In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import folium
import seaborn as sns
from helpers import parse_feature_metadata
import osmnx as ox
import geopandas as gpd

Please set this constant to false if you wish the cells not to plot graphs

In [ ]:
PLOT_GRAPHS = True

Local paths should be correct if repository was cloned

In [ ]:
NEPAL_PATH = r"data\raw_updated_names\nepal_new_feature_names.xlsx"
SENEGAL_PATH = r"data\raw_updated_names\senegal_new_feature_names.xlsx"

nepal_df = pd.read_excel(NEPAL_PATH, index_col=0)
senegal_df = pd.read_excel(SENEGAL_PATH, index_col=0)

## 1. Initial Parsing of the Data
This phase includes removal of features that we from the start knew we would not use or would not be beneficial, and also coerces the remaining features's values to be their declared types

In [ ]:

NEPAL_CAT_COL_NAMES = {'Q11': 'Marital_Status',
 'Q53': 'What_type_of_crop_is_grown_on_this_plot',
 'Q54': 'For_vegetables_what_is_your_source_of_seeds',
 'Q56': 'For_vegetables_do_you_use_seedlings',
 'Q58': 'fertilizer_on_this_plot',
 'Q63': 'What_is_the_main_use_of_produce_from_holding',
 'Q64': 'Do_you_use_machinery_or_and_equipment_on_the_plot',
 'Q65': 'Do_you_do_any_of_the_following',
 'Q67': 'What_do_you_use_soil_analysis_for',
 'Q68': 'How_do_you_conduct_soil_analysis',
 'Q69': 'What_is_correct_for_you',
 'Q71': 'in_the_past_12_months_from_who_did_you_receive_info_on_agriculture',
 'Q73': 'Did_you_receive_anything_from_these_organizations',
 'Q74': 'How_do_you_decide_to_plow',
 'Q75': 'How_do_you_decide_to_begin_sowing',
 'Q76': 'What_type_of_irrigation_do_you_use',
 'Q100': 'Caste',
 'Q105': 'main_sources_of_income',
 'Q111': 'Generally_speaking_would_you_say_that_most_people_can_be_trusted',
 'Q112': 'I_am_much_better_than_most_farmers_here',
 'Q113': 'dislike_not_knowing_what_is_going_to_happen'}

SENEGAL_CAT_COL_NAMES = {'Q63': 'CROP',
 'Q64': 'Seed_source',
 'Q65': 'Variety',
 'Q66': 'Seedlings',
 'Q67': 'Fertilizer',
 'Q72': 'Sold_VEG',
 'Q73': 'Machinery',
 'Q74': 'Practice',
 'Q77': 'Info_source',
 'Q79': 'Did_you_receive_anything_from_the_specified_organizations',
 'Q80': 'Plow_weather',
 'Q81': 'Sow',
 'Q82': 'Irrigation',
 'Q88': 'family_main_sources_income',
 'Q89': 'education_level'}

def fix_column_values(series: pd.Series, meta: dict) -> pd.Series:
    """
    Given a pandas Series and its parsed metadata,
    coerce its values to the right type:
      - continuous, ordinal → numeric (floats or ints; bad → NaN)
      - nominal -> categorical
      - binary → 0/1 integers   (anything non-zero → 1, missing -> NaN)
      - time   → pandas datetime (bad → NaT)
    """
    ftype = meta["type"]
    fname = meta["name"].lower()

    if ftype in ("continuous", "ordinal"):
        # numeric codes or measurements
        return pd.to_numeric(series, errors="coerce")
    if ftype == "nominal":
        return series.astype("category")
    if "binary" in ftype:
        # strings that are not '0': → 1, otherwise __> 0, floats/ints: non-zero → 1, zero or NaN → 0
        def to_binary(x):
            if pd.isna(x):
                return pd.NA
            if isinstance(x, str):
                return x != '0'
            try:
                return 1 if float(x) != 0 else 0
            except:
                return 1
        return series.map(to_binary).astype("Int64")

    if ftype == "time":
        # Force everything to str so we have uniform input
        s = series.astype(str)
        # 0) special: End_Date holds a full datetime → extract only the time
        if "end_date" in fname:
            # parses strings like "4/27/2018 9:47:17 AM" into Timestamps
            dt = pd.to_datetime(s, errors="coerce")
            # grab Python datetime.time
            return dt.dt.time
        # 1) true date column → datetime64
        if "date" in fname:
            return pd.to_datetime(s, format="%Y-%m-%d", errors="coerce")

        # 2) survey‐length durations → timedelta64
        if "length" in fname:
            # strings like "00:21:14" → Timedelta
            return pd.to_timedelta(s, errors="coerce")

        # 3) the two pure clock‐time columns → Python time
        #    strings like "11:27:52" → Timestamp → .time()
        parsed = pd.to_datetime(s, format="%H:%M:%S", errors="coerce")
        return parsed.dt.time

    # otherwise leave it alone
    return series


def clean_survey_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a copy of df in which:
      - all columns named Q<digits>__… have been coerced to their declared types
      - all other columns are left untouched (you can drop them later)
    """
    df = df.copy()
    for col in df.columns:
        meta = parse_feature_metadata(col)
        if meta is None:
            continue
        df[col] = fix_column_values(df[col], meta)
    return df


def drop_non_relevant_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove everything except Q<digits> columns:
    """
    keep = [col for col in df.columns if parse_feature_metadata(col) is not None]
    return df[keep]

import pandas as pd
from collections import defaultdict

def combine_dummy_columns(orig_df: pd.DataFrame, names_dict: dict) -> pd.DataFrame:
    """
    Combines your binary dummy columns into single categorical columns,
    but now *preserves* the option labels.

    For each (qid, type):
      - If there's exactly one dummy → just rename it to end in `-1`.
      - If there are many and it's one-hot (max 1 per row) → new column takes the option LABEL, or None.
      - Otherwise (multi-select) → new column is a list of the LABELS selected (possibly empty).
    """
    df = orig_df.copy()

    # 1) Gather metadata & group columns by (qid, type)
    meta_map   = {}               # col_name → {qid, qname, var_type, dummy_idx, label}
    buckets    = defaultdict(list)

    for col in df.columns:
        if "binary" not in col:
            continue
        m = parse_feature_metadata(col)
        qid        = m["qid"]
        qname      = m["name"]
        var_type   = m["type"]
        dummy_idx  = m["dummy"]
        
        subnames = qname.split(':')
        label      = qname

        meta_map[col] = {
            "qid":       qid,
            "qname":     qname,
            "var_type":  var_type,
            "dummy_idx": dummy_idx,
            "label":     label
        }
        buckets[(qid, var_type)].append(col)

    to_drop  = []
    new_cols = {}

    # 2) Process each question group
    for (qid, var_type), cols in buckets.items():
        # sort by dummy_idx so labels stay in the right order
        cols_sorted   = sorted(cols, key=lambda c: meta_map[c]["dummy_idx"])
        labels_sorted = [meta_map[c]["label"] for c in cols_sorted]
        arr           = df[cols_sorted].fillna(0).astype(int).values

        # A) Single-dummy → just rename it to ...-1
        if len(cols_sorted) == 1:
            old = cols_sorted[0]
            new = f"{qid}__{meta_map[old]['qname']}__binary__1"
            df = df.rename(columns={old: new})
            continue
        

        new_type = "nominal" if var_type.endswith("nominal") else "ordinal"
        qname = names_dict[qid]
        new_name = f"{qid}__{qname}__{new_type}"
        

        # B) One-hot? (no row has more than one “1”)
        if arr.sum(axis=1).max() <= 1:
            cat = []
            for row in arr:
                if row.sum() == 0:
                    cat.append(None)
                else:
                    cat.append(labels_sorted[row.argmax()])
            
            new_cols[new_name] = cat

        # C) Multi-select
        else:
            multi = []
            for row in arr:
                # collect all the labels whose dummy==1
                sel = [labels_sorted[i] for i, v in enumerate(row) if v == 1]
                multi.append(tuple(sel))
            new_cols[new_name] = multi

        to_drop.extend(cols_sorted)

    # 3) Drop old dummies & add the new columns
    if to_drop:
        df = df.drop(columns=to_drop)
        df = pd.concat([df, pd.DataFrame(new_cols, index=df.index)], axis=1)

    return df, new_cols



In [ ]:
# dropping non Q#... columns
nepal_df = drop_non_relevant_columns(nepal_df)
senegal_df = drop_non_relevant_columns(senegal_df)

# initial clean
nepal_df = clean_survey_dataframe(nepal_df)
senegal_df = clean_survey_dataframe(senegal_df)

# combine and remove dummy columns
nepal_df, npl_new_cols = combine_dummy_columns(nepal_df, NEPAL_CAT_COL_NAMES)
senegal_df, sng_new_cols = combine_dummy_columns(senegal_df, SENEGAL_CAT_COL_NAMES)

# Univariate Analysis

## Analyze Datetime columns

Create mew datetime column

In [ ]:


nepal_df['Q2__SurveyedDate__time'] = pd.to_datetime(nepal_df['Q2__SurveyedDate__time'], errors='coerce')

def create_datetime_col(df):
    # ensure your date column is datetime64
    df['Q2__SurveyedDate__time'] = pd.to_datetime(
        df['Q2__SurveyedDate__time'], format="%Y-%m-%d", errors='coerce'
    )

    # 1) build an all‐string “HH:MM:SS” series
    time_str = df['Q2__SurveyedTime__time'].astype(str).str.extract(r'(\d{2}:\d{2}:\d{2})')[0]

    # 2) combine
    dt_strings = df['Q2__SurveyedDate__time'].dt.strftime('%Y-%m-%d') + ' ' + time_str

    # 3) parse into real timestamps
    df['Q2__Surveyed_Date_Time__time'] = pd.to_datetime(dt_strings, errors='coerce')

    print(df['Q2__Surveyed_Date_Time__time'].dtype)  # should be datetime64[ns]
    
create_datetime_col(nepal_df)
create_datetime_col(senegal_df)

Removed old one since already have DateTime column

In [ ]:

nepal_df = nepal_df.drop(['Q2__SurveyedDate__time', 'Q2__SurveyedTime__time'], axis=1)
senegal_df = senegal_df.drop(['Q2__SurveyedDate__time', 'Q2__SurveyedTime__time'], axis=1)

Define function to analyze new datetime columns

In [ ]:

def analyze_datetime(df, col):
    """
    Perform univariate analysis on a datetime64 column:
      - prints basic stats (min, max, missing)
      - plots counts per day, month, weekday, and hour-of-day
    """
    # Basic info
    series = df[col]
    if not series.dtype.name.startswith('datetime'):
        raise ValueError(f"Column {col!r} is not datetime dtype.")
    
    print(f"\n--- ANALYSIS OF {col.upper()} ---")
    print(f"Type: {series.dtype}")
    print(f"Missing: {series.isna().sum()} / {len(series)}")
    print(f"Range: {series.min()} → {series.max()}")
    
    # Extract time features
    df_temp = df.dropna(subset=[col]).copy()
    df_temp['date']    = df_temp[col].dt.date
    df_temp['month']   = df_temp[col].dt.to_period('M').astype(str)
    df_temp['weekday'] = df_temp[col].dt.day_name()
    df_temp['hour']    = df_temp[col].dt.hour
    
    # 1. Counts per day
    daily = df_temp.groupby('date').size()
    plt.figure()
    daily.plot(kind='bar')
    plt.title(f"Number of Records per Day ({col})")
    plt.xlabel("Date")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 2. Counts by month
    monthly = df_temp.groupby('month').size().sort_index()
    plt.figure()
    monthly.plot(kind='bar')
    plt.title(f"Counts by Month ({col})")
    plt.xlabel("Month")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 3. Counts by weekday
    # ensure Mon–Sun order
    weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    wd = df_temp['weekday'].value_counts().reindex(weekdays)
    plt.figure()
    wd.plot(kind='bar')
    plt.title(f"Counts by Weekday ({col})")
    plt.xlabel("Weekday")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()
    
    # 4. Distribution of hour of day
    plt.figure()
    sns.histplot(df_temp['hour'], bins=24, discrete=True, kde=False)
    plt.title(f"Survey Time of Day Distribution ({col})")
    plt.xlabel("Hour of Day")
    plt.ylabel("Count")
    plt.xticks(range(0,24))
    plt.tight_layout()
    plt.show()

    

Analyze new datetime columns

In [ ]:
if PLOT_GRAPHS: 
    print("NEPAL DATA")
    analyze_datetime(nepal_df, 'Q2__Surveyed_Date_Time__time')
    print("SENEGAL DATA")
    analyze_datetime(senegal_df, 'Q2__Surveyed_Date_Time__time')

## Analyze Survey Length

Add Survey_Length column

In [ ]:

# turn time → "HH:MM:SS" strings → Timedelta
nepal_df['Q0__SurveyLengthTime__time'] = (
    nepal_df['Q0__SurveyLengthTime__time']
      .astype(str)                # datetime.time → "HH:MM:SS"
      .pipe(pd.to_timedelta)      # parse into timedelta64[ns]
)

# first, make sure both your start‐ and end‐times are strings
senegal_df['Q2__Surveyed_Date_Time__time'] = senegal_df['Q2__Surveyed_Date_Time__time'].astype(str)
senegal_df['Q0__Surveyed_End_Date__time']   = senegal_df['Q0__Surveyed_End_Date__time'].astype(str)

# 1) Parse your “start” column as a real timestamp
senegal_df['start_dt'] = pd.to_datetime(
    senegal_df['Q2__Surveyed_Date_Time__time'],
    format='%Y-%m-%d %H:%M:%S'
)

# 2) Turn your “end time” (just HH:MM:SS) into a Timedelta
senegal_df['end_td'] = pd.to_timedelta(
    senegal_df['Q0__Surveyed_End_Date__time'].astype(str)
)

# 3) Build a full “end” timestamp by taking the date from start_dt and adding the time delta
senegal_df['end_dt'] = (
    senegal_df['start_dt'].dt.normalize()  # midnight of the start date
  + senegal_df['end_td']
)

senegal_df['Q0__SurveyLengthTime__time'] = senegal_df['end_dt'] - senegal_df['start_dt']

senegal_df.drop(columns=['start_dt', 'end_td', 'end_dt'], inplace=True)

Function to analyze Survey_Length column

In [ ]:
from matplotlib.ticker import FuncFormatter

def analyze_survey_length(df, column='Q0__SurveyLengthTime__time', bins=30):
    """
    Analyze the distribution of survey lengths in a DataFrame.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing the survey length column.
    column : str, default 'Survey_Length'
        Name of the timedelta column to analyze.
    bins : int, default 30
        Number of histogram bins.

    Outputs
    -------
    - Histogram of survey lengths (HH:MM:SS) distribution.
    - Printed mean and standard deviation of survey lengths.
    - Returns a tuple (mean_timedelta, std_timedelta).
    """
    # Ensure the column is timedelta64[ns]
    if not pd.api.types.is_timedelta64_dtype(df[column]):
        raise TypeError(f"Column '{column}' must be timedelta64[ns] dtype")

    # Compute statistics
    mean_td = df[column].mean()
    std_td = df[column].std()

    # Convert to total seconds for plotting
    secs = df[column].dt.total_seconds()

    # Plot histogram
    fig, ax = plt.subplots()
    ax.hist(secs, bins=bins)
    ax.set_title("Distribution of Survey Lengths")
    ax.set_xlabel("Survey Length (HH:MM:SS)")
    ax.set_ylabel("Frequency")

    # Formatter to convert seconds back to HH:MM:SS
    def sec_to_hhmmss(x, pos):
        h = int(x // 3600)
        m = int((x % 3600) // 60)
        s = int(x % 60)
        return f"{h:02d}:{m:02d}:{s:02d}"

    ax.xaxis.set_major_formatter(FuncFormatter(sec_to_hhmmss))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Print mean and std in HH:MM:SS
    mean_str = str(mean_td).split()[-1]
    std_str  = str(std_td).split()[-1]
    print(f"Mean survey length: {mean_str}")
    print(f"Std dev survey length: {std_str}")

    return mean_td, std_td

In [ ]:
if PLOT_GRAPHS:
    print("NEPAL DATA")
    analyze_survey_length(nepal_df)
    print("SENEGAL DATA")
    analyze_survey_length(senegal_df)

Discovered data-entry glitch extreme value at one Senegalese survey

In [ ]:
bad = senegal_df[ senegal_df['Q0__SurveyLengthTime__time'] < pd.Timedelta(0) ]
print(bad[['Q2__Surveyed_Date_Time__time','Q0__Surveyed_End_Date__time','Q0__SurveyLengthTime__time']])

Filling in this survey with median survey length

In [ ]:
median_td = senegal_df.loc[ senegal_df['Q0__SurveyLengthTime__time'] >= pd.Timedelta(0), 'Q0__SurveyLengthTime__time' ].median()
mask = senegal_df['Q0__SurveyLengthTime__time'] < pd.Timedelta(0)
print(f"Found {mask.sum()} negative Survey_Length(s), filling with median = {median_td}")
senegal_df.loc[mask, 'Q0__SurveyLengthTime__time'] = median_td

Redefine survey analysis function by adding standard deviation markers

In [ ]:
from matplotlib.ticker import FuncFormatter

def analyze_survey_length(df, column='Q0__SurveyLengthTime__time', bins=30, max_std=3):
    """
    Analyze the distribution of survey lengths in a DataFrame,
    and overlay vertical lines at mean ± n * std for n = 1..max_std.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing the survey length column.
    column : str, default 'Survey_Length'
        Name of the timedelta column to analyze.
    bins : int, default 30
        Number of histogram bins.
    max_std : int, default 3
        How many standard-deviation multiples to draw lines for.

    Outputs
    -------
    - Histogram of survey lengths (HH:MM:SS) distribution with σ-lines.
    - Printed mean and standard deviation of survey lengths.
    - Returns a tuple (mean_timedelta, std_timedelta).
    """
    # 1) Type check
    if not pd.api.types.is_timedelta64_dtype(df[column]):
        raise TypeError(f"Column '{column}' must be timedelta64[ns] dtype")

    # 2) Compute statistics
    mean_td = df[column].mean()
    std_td  = df[column].std()

    mean_sec = mean_td.total_seconds()
    std_sec  = std_td.total_seconds()

    # 3) Build histogram
    secs = df[column].dt.total_seconds()
    fig, ax = plt.subplots(figsize=(8,4))
    ax.hist(secs, bins=bins, edgecolor='black', alpha=0.7)
    ax.set_title("Distribution of Survey Lengths")
    ax.set_xlabel("Survey Length (HH:MM:SS)")
    ax.set_ylabel("Frequency")

    # Formatter: seconds → HH:MM:SS
    def sec_to_hhmmss(x, pos):
        h = int(x // 3600)
        m = int((x % 3600) // 60)
        s = int(x % 60)
        return f"{h:02d}:{m:02d}:{s:02d}"

    ax.xaxis.set_major_formatter(FuncFormatter(sec_to_hhmmss))
    plt.xticks(rotation=45)

    # 4) Overlay mean and std-bands
    #    use different linestyles/colors for clarity
    for n in range(0, max_std+1):
        # skip n=0 since that’s the mean line (draw it separately for legend clarity)
        if n == 0:
            ax.axvline(mean_sec, color='black', linestyle='-', linewidth=2,
                       label='Mean')
        else:
            for sign, label in [(+1, f'+{n}σ'), (-1, f'-{n}σ')]:
                x = mean_sec + sign * n * std_sec
                ax.axvline(x, linestyle='--' if n==1 else ':',
                           linewidth=1.5 if n==1 else 1,
                           label=label if sign>0 else None)
    # Only one -σ label needed, so we skip labeling the negative side

    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

    # 5) Print out
    mean_str = str(mean_td).split()[-1]
    std_str  = str(std_td).split()[-1]
    print(f"Mean survey length: {mean_str}")
    print(f"Std dev survey length: {std_str}")

    return mean_td, std_td

In [ ]:
if PLOT_GRAPHS:
    print("SENEGAL DATA")
    analyze_survey_length(senegal_df)

Outliers defined as beyond 3 standard deviations, we swap outliers with median survey length

In [ ]:
threshold = senegal_df['Q0__SurveyLengthTime__time'].mean() + 2*senegal_df['Q0__SurveyLengthTime__time'].std()
mask = senegal_df['Q0__SurveyLengthTime__time'] > threshold
print(f"Changed {mask.sum()} outlier survey lengths to median")
senegal_df.loc[mask, 'Q0__SurveyLengthTime__time'] = median_td

Recheck survey length histogram

In [ ]:
if PLOT_GRAPHS:
    print("SENEGAL DATA")
    analyze_survey_length(senegal_df)

## Analyze Locational Data

Since added Survey_Length column, took out others

In [ ]:

nepal_df = nepal_df.drop(['Q0__SurveyedEndTime__time'], axis=1)
senegal_df = senegal_df.drop(['Q0__Surveyed_End_Date__time'], axis=1)

Showing missing columns in the Senegal dataset's Location_... columns

In [ ]:
for column in ['Latitude', 'Longitude', 'Altitude', 'Accuracy']:
    print(f"missing values in Q1__Location_{column}: {senegal_df[f'Q1__Location_{column}__continuous'].isna().sum()}")
    print(f"missing values in {column}: {senegal_df[f'Q1__{column}__continuous'].isna().sum()}")


Removing Location_ columns from senegal data


In [ ]:
senegal_df = senegal_df.drop(['Q1__Location_Latitude__continuous', 'Q1__Location_Longitude__continuous', 'Q1__Location_Altitude__continuous', 'Q1__Location_Accuracy__continuous'], axis=1)

Both are ok for nepal

In [ ]:
for column in ['Latitude', 'Longitude', 'Altitude', 'Accuracy']:
    print(f"missing values in Location_{column}: {nepal_df[f'Q1__Location_{column}__continuous'].isna().sum()}")
    print(f"missing values in {column}: {nepal_df[f'Q1__{column}__continuous'].isna().sum()}")

Still removing from nepal Location_ columns for consistency between both datasets

In [ ]:
nepal_df = nepal_df.drop(['Q1__Location_Latitude__continuous', 'Q1__Location_Longitude__continuous', 'Q1__Location_Altitude__continuous', 'Q1__Location_Accuracy__continuous'], axis=1)

Defining function to analyze locational columns

In [ ]:

from folium.plugins import MarkerCluster

def lat_lon_hillshade_map(df, zoom_start=11):
    """
    Folium map with:
      • Default OpenStreetMap layer
      • ESRI World Hillshade (true terrain) layer
      • MarkerCluster of your points with Altitude pop-ups
      • LayerControl for toggling back and forth
    """
    # 1) Prepare & center
    df_map = df.dropna(subset=['Q1__Latitude__continuous','Q1__Longitude__continuous']).copy()
    center = [df_map['Q1__Latitude__continuous'].mean(), df_map['Q1__Longitude__continuous'].mean()]

    # 2) Build base map (OSM by default)
    m = folium.Map(location=center, zoom_start=zoom_start)

    # 3) Add ESRI World Hillshade as a second base layer
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/'
              'Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}',
        attr='Tiles © Esri — Source: USGS',
        name='Hillshade (ESRI)'
    ).add_to(m)

    # 4) Your markers (clustered)
    mc = MarkerCluster(name='Points').add_to(m)
    for idx, row in df_map.iterrows():
        folium.Marker(
            location=[row['Q1__Latitude__continuous'], row['Q1__Longitude__continuous']],
            popup=(
                f"<b>Index:</b> {idx}<br>"
                f"<b>Altitude:</b> {row.get('Q1__Altitude__continuous','N/A')} m"
            )
        ).add_to(mc)

    # 5) Toggle control
    folium.LayerControl(collapsed=False).add_to(m)

    return m


Analyze lat long data for nepal, both openstreetmap and hillshade for elevation

In [ ]:
if PLOT_GRAPHS:
    nepal_map = lat_lon_hillshade_map(nepal_df)
    nepal_map   # in Jupyter this will render with two base‐layers to switch between


The same for Senegal

In [ ]:
if PLOT_GRAPHS:
    print("SENEGAL DATA")
    senegal_map = lat_lon_hillshade_map(senegal_df)
    senegal_map


Check against google if the altitude column is accurate given the lat lot coordinates, plot accuracy also to see if there is some connection

In [ ]:
import requests

def plot_altitude_error(df, lat_col='Q1__Latitude__continuous', lon_col='Q1__Longitude__continuous', alt_col='Q1__Altitude__continuous', accuracy_col='Q1__Accuracy__continuous', api_url='https://api.open-elevation.com/api/v1/lookup', batch_size=100, bins=30):
    '''
    Fetches true elevations and plots a combined view showing:
      - Histogram of altitude errors (recorded - true).
      - Median reported accuracy for each error bin overlaid as a line.

    Parameters:
    df (pd.DataFrame): DataFrame with latitude, longitude, recorded altitude, and accuracy columns.
    lat_col (str): Name of the latitude column.
    lon_col (str): Name of the longitude column.
    alt_col (str): Name of the recorded altitude column.
    accuracy_col (str): Name of the accuracy column.
    api_url (str): Elevation API endpoint.
    batch_size (int): Number of locations per API request.
    bins (int): Number of bins for error histogram and averaging.

    Returns:
    None. Displays a joint plot of error distribution and median accuracy per error bin.
    '''
    # Work on a copy
    df = df.copy()
    true_alts = []

    # Batch request loop to fetch true elevations
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        locations = '|'.join(f"{lat},{lon}" for lat, lon in zip(batch[lat_col], batch[lon_col]))
        resp = requests.get(api_url, params={'locations': locations})
        resp.raise_for_status()
        results = resp.json().get('results', [])
        if len(results) != len(batch):
            raise ValueError(f"Expected {len(batch)} results, got {len(results)}")
        true_alts.extend(r['elevation'] for r in results)

    # Compute errors and extract accuracy
    recorded = df[alt_col].values
    errors = recorded - np.array(true_alts)
    accuracies = df[accuracy_col].values

    # Prepare bins and compute median accuracy per bin
    bin_edges = np.linspace(errors.min(), errors.max(), bins + 1)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    digitized = np.digitize(errors, bin_edges) - 1
    median_acc = [np.median(accuracies[digitized == i]) if np.any(digitized == i) else np.nan for i in range(bins)]

    # Create combined plot
    fig, ax1 = plt.subplots()

    # Histogram of errors
    ax1.hist(errors, bins=bin_edges, alpha=0.6)
    ax1.set_xlabel('Altitude Error (m)')
    ax1.set_ylabel('Frequency')
    ax1.axvline(0, color='k', linestyle='--')


    plt.title('Altitude Error Distribution')
    plt.show()

In [ ]:
if PLOT_GRAPHS:
    plot_altitude_error(nepal_df)

In [ ]:
if PLOT_GRAPHS:
    plot_altitude_error(senegal_df)

Looks like gaussian noise error, adding approximate average to center errors 

In [ ]:
nepal_df["Q1__Altitude__continuous"] = nepal_df["Q1__Altitude__continuous"] + 62
senegal_df["Q1__Altitude__continuous"] = senegal_df["Q1__Altitude__continuous"] - 22

In [ ]:
if PLOT_GRAPHS:
    plot_altitude_error(nepal_df)

In [ ]:
if PLOT_GRAPHS:
    plot_altitude_error(senegal_df)

# Missing Values

View missing counts

In [ ]:
missing_counts_nepal = nepal_df.isnull().sum()
print("Missing values per column in nepal:")
print(missing_counts_nepal[missing_counts_nepal>0])


In [ ]:
missing_counts_senegal = senegal_df.isnull().sum()
print("Missing values per column in senegal:")
print(missing_counts_senegal[missing_counts_senegal>0])



Drop almost completely missing columns in Senegal

In [ ]:
senegal_df = senegal_df.drop(columns=["Q0__Survey_duration__time", "Q89__education_level__nominal"], axis=1)

We remove these columns because our coordinator said we should

In [ ]:
children_cols = ['Q102__How_many_children_do_you_have__continuous', 'Q103__How_many_children_do_you_plan_to_have_in_your_whole_live__continuous']

nepal_df = nepal_df.drop(columns=children_cols, axis=1)


Seperate numeric and categorical columns

In [ ]:
nepal_missing_cols = nepal_df.columns[nepal_df.isnull().any() == True]


nepal_num_cols = [c for c in nepal_df.columns if parse_feature_metadata(c)["type"] == "continuous"]
nepal_cat_cols = [c for c in nepal_df.columns if parse_feature_metadata(c)["type"] not in ["continuous", "time"]]
nepal_num_missing_cols = [c for c in nepal_missing_cols if c not in nepal_cat_cols]


senegal_missing_cols = senegal_df.columns[senegal_df.isnull().any() == True]

senegal_num_cols = [c for c in senegal_df.columns if parse_feature_metadata(c)["type"] == "continuous"]
senegal_cat_cols = [c for c in senegal_df.columns if parse_feature_metadata(c)["type"] not in ["continuous", "time"]]
senegal_num_missing_cols = [c for c in senegal_missing_cols if c not in senegal_cat_cols]



We use MICE imputation to deal with missing numeric values in both datasets, and impute missing categorical values with the mode of that column

In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer, SimpleImputer

mice_imp = mice_imp = IterativeImputer(
    random_state=123,
    sample_posterior=True,    # draws from the posterior → multiple imputation flavour
    max_iter=10,
    n_nearest_features=5
)

imp_n_df = nepal_df.copy()
imp_s_df = senegal_df.copy()

imp_n_df[nepal_num_cols] = mice_imp.fit_transform(imp_n_df[nepal_num_cols])
imp_s_df[senegal_num_cols] = mice_imp.fit_transform(imp_s_df[senegal_num_cols])

mode_imp = SimpleImputer(strategy="most_frequent")

# turn every Python None into a real NaN in your categorical block
imp_n_df[nepal_cat_cols] = imp_n_df[nepal_cat_cols] \
    .replace({None: np.nan})

imp_n_df.loc[:, nepal_cat_cols] = mode_imp.fit_transform(imp_n_df[nepal_cat_cols])
imp_s_df.loc[:, senegal_cat_cols] = mode_imp.fit_transform(imp_s_df[senegal_cat_cols])

Check that numeric column imputations are plausible

In [ ]:
for col in nepal_num_missing_cols:
    sns.kdeplot(nepal_df[col].dropna(), label="obs")
    sns.kdeplot(imp_n_df[col], label="imp")
    plt.legend(); plt.title(col); plt.show()


In [ ]:
for col in senegal_num_missing_cols:
    sns.kdeplot(senegal_df[col].dropna(), label="obs")
    sns.kdeplot(imp_s_df[col], label="imp")
    plt.legend(); plt.title(col); plt.show()

Apply to original datasets

In [ ]:
nepal_df = imp_n_df
senegal_df = imp_s_df

Recheck missing values in both datasets

In [ ]:
missing_counts_nepal = nepal_df.isnull().sum()
print("Missing values per column in nepal:")
print(missing_counts_nepal[missing_counts_nepal>0])

In [ ]:
missing_counts_senegal = senegal_df.isnull().sum()
print("Missing values per column in senegal:")
print(missing_counts_senegal[missing_counts_senegal>0])

# Outliers

First visualize the distributions of numeric columns with boxplots and histograms and also view their skew metrics and how many IQR-based outliers are detected

In [ ]:
from scipy.stats import skew        # skewness metric


def visualize_num_columns(df, num_cols, country):
    for col in num_cols:
        series = df[col].dropna()

        # ── 1. Visualise ──────────────────────────────────────────────────────────
        fig, ax = plt.subplots(1, 2, figsize=(10, 4))
        sns.boxplot(y=series, ax=ax[0])
        ax[0].set_title(f"{col} • Boxplot")

        sns.histplot(series, kde=True, ax=ax[1])
        ax[1].set_title(f"{col} • Histogram / KDE")

        plt.suptitle(f"{country}: {col}", fontsize=11, y=1.02)
        plt.tight_layout()
        plt.show()

        # ── 2. Skewness ──────────────────────────────────────────────────────────
        skew_val = skew(series)
        print(f"  • Skewness of '{col}': {skew_val:+.3f}")

        # ── 3. IQR-based outlier detection ──────────────────────────────────────
        Q1, Q3 = series.quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        mask_outliers = (series < lower) | (series > upper)
        n_outliers = mask_outliers.sum()

        print(f"  • IQR outliers in '{col}': {n_outliers} "
              f"(lower < {lower:.3f}, upper > {upper:.3f})")

In [ ]:


# Pair each DataFrame with its list of numeric columns and a label
dfs_and_cols = [
    (nepal_df,   nepal_num_cols,   "Nepal"),
    (senegal_df, senegal_num_cols, "Senegal")
]

for df, num_cols, country in dfs_and_cols:
    print(f"\nEvaluating outliers in numerical columns of the {country} dataset")

    visualize_num_columns(df, num_cols, country)
    


Avoided as much as possible winsorization, applied log transformations when skew was over 1.0, between 0.5 and 1.0 applied sqrt transformation, below 0.5 left as is.
Will use bicor or spearman correlation in EFA to mitigate effect of remaining outliers

In [ ]:

from scipy.stats import boxcox

df = nepal_df.copy()

def new_name(col, tag):
    """Insert _log or _sqrt right before '__continuous'."""
    return col.replace("__continuous", f"{tag}__continuous")

# signed-log1p for variables that can be negative
signed_log1p = lambda x: np.sign(x) * np.log1p(np.abs(x))

# 1) Altitude  ────────────────────────────────────────────────────────────
col = "Q1__Altitude__continuous"
upper = df[col].quantile(0.99)
df[new_name(col, "_log")] = np.log1p(df[col].clip(lower=0, upper=upper))
df.drop(columns=[col], inplace=True)

# 2) Rented land (bigha)  ────────────────────────────────────────────────
col = "Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha__continuous"
df[new_name(col, "_log")] = np.log1p(df[col])

ord_name = col.replace("__continuous", "__ordinal")
df[ord_name] = pd.cut(
    df[col],
    bins=[-np.inf, 0, 1.5, np.inf],
    labels=["none", "little", "much"],
    include_lowest=True,
    ordered=True
)

# Drop the original continuous column
df.drop(columns=[col], inplace=True)


# 3) Vegetable plot size (bigha)  ────────────────────────────────────────
col = "Q52__On_how_much_land_do_you_grow_vegetables_bigha__continuous"
df[new_name(col, "_log")] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# 4) Vegetable harvest (kg)  ─────────────────────────────────────────────
col = "Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms__continuous"
df[new_name(col, "_log")] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# 5) Drip-irrigation years → binary  ─────────────────────────────────────
col = "Q78__How_many_years_have_you_been_using_drip_irrigation_if_no_zero__continuous"
df[col.replace("__continuous", "__binary__1")] = (df[col] > 0).astype(int)
df.drop(columns=[col], inplace=True)

# 6) Household size  ─────────────────────────────────────────────────────
col = "Q101__how_many_people_live_in_this_household__continuous"
df[new_name(col, "_log")] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# 7) Wife/husband education years  ───────────────────────────────────────
col = "Q107__Education_level_of_your_wife_husband_How_many_years_of_formal_education__continuous"
df[new_name(col, "_log")] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# 8) Agriculture income (remove 1 extreme, then log)  ────────────────────
col = "Q108__What_is_your_households_yearly_income_from_agriculture_NPR__continuous"
# new: remove the row with the extreme value entirely
idx = df[col].idxmax()
df.drop(index=idx, inplace=True)
# log transform
df[new_name(col, "_log")] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# 9) Overall income  ─────────────────────────────────────────────────────
col = "Q109__What_is_your_households_yearly_income_overall_including_agriculture_NPR__continuous"
df[new_name(col, "_log")] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# 10) Loan amount  ───────────────────────────────────────────────────────
col = "Q110__Do_you_currently_have_a_loan_if_no_zero_if_yes_amount_of_money_loaned_NPR__continuous"
ord_name = col.replace("__continuous", "__ordinal")
df[ord_name] = pd.cut(
    df[col],
    bins=[-np.inf, 0, 1, np.inf],
    labels=["none", "little", "much"],
    include_lowest=True,
    ordered=True
)

# Drop the original continuous column
df.drop(columns=[col], inplace=True)

# 11) Remove coordinator-flagged totals  ─────────────────────────────────
df.drop(columns=[
    "Q0__positive_total__continuous",
    "Q0__negative_total__continuous"
], errors="ignore", inplace=True)

# 12) Distance  ──────────────────────────────────────────────────────────
col = "Q0__Distance__continuous"
df[new_name(col, "_log")] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# 13) Total area (sqrt)  ─────────────────────────────────────────────────
col = "Q0__TOTAL_AREA__continuous"
df[new_name(col, "_sqrt")] = np.sqrt(df[col])
df.drop(columns=[col], inplace=True)

# df now holds the transformed / recoded columns with correct naming


In [ ]:
num_cols = []
for col in df:
    if "__continuous" in col:
        num_cols.append(col)
    
visualize_num_columns(df, num_cols, "Nepal")

In [ ]:
nepal_df = df

Used light winsorization in specific cases, applied log transformations when skew was over 1.0, between 0.5 and 1.0 applied sqrt transformation, below 0.5 left as is. Will use bicor or spearman correlation in EFA to mitigate effect of remaining outliers

In [ ]:
import numpy as np
import pandas as pd

# work on a copy
df = senegal_df.copy()

# ── helper functions ─────────────────────────────────────────────────────────
def rename_cont(col, tag):
    """
    Insert a tag (_log, _sqrt) just before the '_continuous' suffix.
    """
    if col.endswith("__continuous"):
        return col.replace("__continuous", f"{tag}__continuous")
    elif col.endswith("_continuous"):
        return col.replace("_continuous", f"{tag}_continuous")
    else:
        raise ValueError(f"Column {col} does not end with '_continuous'")

def tukey_fence(series):
    """
    Compute lower and upper Tukey fences for a pandas Series.
    Returns (lower, upper).
    """
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    return q1 - 1.5 * iqr, q3 + 1.5 * iqr

# ── 1. Q1_Altitude_continuous: winsorize at Tukey fences ────────────────────
col = "Q1__Altitude__continuous"
low, high = tukey_fence(df[col].dropna())
df[col] = df[col].clip(lower=low, upper=high)

# ── 2. Q0__Hope_total__continuous: clip upper tail at 54 ────────────────────
col = "Q0__Hope_total__continuous"
df[col] = df[col].clip(upper=54)

# ── 3. Q0__Average_CS__continuous: clip upper tail at 4.5 ──────────────────
col = "Q0__Average_CS__continuous"
df[col] = df[col].clip(upper=4.5)

# ── 4. Q61__Land_rented_cultivated_ha__continuous: log1p transform ─────────
col = "Q61__Land_rented_cultivated_ha__continuous"
new = rename_cont(col, "_log")
df[new] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# ── 5. Q71__VEG_harvest_per_year_kg__continuous: log1p transform ───────────
col = "Q71__VEG_harvest_per_year_kg__continuous"
new = rename_cont(col, "_log")
df[new] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# ── 6. Q84__Years_using_drip_irrigation__continuous: log1p transform ───────
col = "Q84__Years_using_drip_irrigation__continuous"
new = rename_cont(col, "_log")
df[new] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# ── 7. Q87__Household_size__continuous: sqrt transform ─────────────────────
col = "Q87__Household_size__continuous"
new = rename_cont(col, "_sqrt")
df[new] = np.sqrt(df[col])
df.drop(columns=[col], inplace=True)

# ── 8. Q0__Education_years__continuous: log1p transform ────────────────────
col = "Q0__Education_years__continuous"
new = rename_cont(col, "_log")
df[new] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# ── 9. Q90__Yearly_income_agriculture_XOF__continuous:
#     drop the two largest outliers, then log1p transform ───────────────────
col = "Q90__Yearly_income_agriculture_XOF__continuous"
# identify and drop the two rows with the highest values
outliers = df[col].nlargest(2).index
df.drop(index=outliers, inplace=True)
# now transform
new = rename_cont(col, "_log")
df[new] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# ── 10. Q91__Yearly_income_overall_XOF__continuous: log1p transform ─────────
col = "Q91__Yearly_income_overall_XOF__continuous"
new = rename_cont(col, "_log")
df[new] = np.log1p(df[col])
df.drop(columns=[col], inplace=True)

# ── Q92: Current loan amount ────────────────────────────────────────────────
col = "Q92__Current_loan_amount_XOF__continuous"

# Create a 3‐level ordinal band: 
#    - 'near'   = no loan (0 XOF)
#    - 'medium' = loan > 0 up to the median of positive loans
#    - 'far'    = loan above that median
ord_name = col.replace("__continuous", "__ordinal")
df[ord_name] = pd.cut(
    df[col],
    bins=[-np.inf, 0, 1e6, np.inf],
    labels=["near", "medium", "far"],
    include_lowest=True,
)

# 3) Drop the original continuous column
df.drop(columns=[col], inplace=True)

# ── 12. Q0__Distance_Dakar_KM__continuous: sqrt transform ──────────────────
col = "Q0__Distance_Dakar_KM__continuous"
new = rename_cont(col, "_sqrt")
df[new] = np.sqrt(df[col])
df.drop(columns=[col], inplace=True)

# ── 13. Q0__Age_Education_interaction__continuous: winsorize ───────────────
col = "Q0__Age_Education_interaction__continuous"
low, high = tukey_fence(df[col].dropna())
df[col] = df[col].clip(lower=low, upper=high)

# ── 'df' now contains your Senegal data with all requested transforms ───────


In [ ]:
num_cols = []
for col in df:
    if "__continuous" in col:
        num_cols.append(col)
    
visualize_num_columns(df, num_cols, "Senegal")

In [ ]:
senegal_df = df

# Create Productivity Metric Columns (Targets)

In [ ]:
def standardize(x):
    return (x - x.mean()) / x.std()

In [ ]:

NPL_YEARLY_AGR_INCOME_COL = "Q108__What_is_your_households_yearly_income_from_agriculture_NPR_log__continuous"
SNGL_YEARLY_AGR_INCOME_COL = 'Q90__Yearly_income_agriculture_XOF_log__continuous'
NPL_OWND_CULTVTD_LAND_COL = "Q50__How_much_land_that_is_yours_do_you_cultivate_bigha__continuous"
SNGL_OWND_CULTVTD_LAND_COL = "Q60__Land_owned_cultivated_ha__continuous"
NPL_LEAS_CULTVTD_LAND_COL = "Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha_log__continuous"
SNGL_LEAS_CULTVTD_LAND_COL = "Q61__Land_rented_cultivated_ha_log__continuous"
NPL_VEG_HARVEST_PER_YEAR_COL = "Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms_log__continuous"
SNGL_VEG_HARVEST_PER_YEAR_COL = "Q71__VEG_harvest_per_year_kg_log__continuous"
NPL_SELF_REPORTED_FARM_LEVEL_COL = "Q112__Generally_speaking_how_would_you_define_your_farming__ordinal"
SNGL_SELF_REPORTED_FARM_LEVEL_COL = "Q94__Farming_level_relative__ordinal"
NPL_VEG_LAND = "Q52__On_how_much_land_do_you_grow_vegetables_bigha_log__continuous"
SNGL_VEG_LAND = "Q62__Land_grow_vegetables_ha__continuous"

nepal_df['yearly_income_from_agr_per_land_Z'] = ((nepal_df[NPL_YEARLY_AGR_INCOME_COL].transform(standardize)) / (nepal_df[NPL_OWND_CULTVTD_LAND_COL].transform(standardize) + nepal_df[NPL_LEAS_CULTVTD_LAND_COL].transform(standardize))).transform(standardize)
senegal_df['yearly_income_from_agr_per_land_Z'] = ((senegal_df[SNGL_YEARLY_AGR_INCOME_COL].transform(standardize)) / (senegal_df[SNGL_OWND_CULTVTD_LAND_COL].transform(standardize) + senegal_df[SNGL_LEAS_CULTVTD_LAND_COL].transform(standardize))).transform(standardize)

nepal_df['yearly_income_from_agr_Z'] = nepal_df[NPL_YEARLY_AGR_INCOME_COL].transform(standardize)
senegal_df['yearly_income_from_agr_Z'] = senegal_df[SNGL_YEARLY_AGR_INCOME_COL].transform(standardize)

nepal_df['veg_per_area_Z'] = (nepal_df[NPL_VEG_HARVEST_PER_YEAR_COL].transform(standardize) / nepal_df[NPL_VEG_LAND].transform(standardize)).transform(standardize)
senegal_df['veg_per_area_Z'] = (senegal_df[SNGL_VEG_HARVEST_PER_YEAR_COL].transform(standardize) / senegal_df[SNGL_VEG_LAND].transform(standardize)).transform(standardize)

nepal_df['self_farming_perception_Z'] = nepal_df[NPL_SELF_REPORTED_FARM_LEVEL_COL].transform(standardize)
senegal_df['self_farming_perception_Z'] = senegal_df[SNGL_SELF_REPORTED_FARM_LEVEL_COL].transform(standardize)

# defragment
nepal_df = nepal_df.copy()
senegal_df = senegal_df.copy()

target_cols = ['yearly_income_from_agr_per_land_Z',
               'yearly_income_from_agr_Z',
               'veg_per_area_Z',
               'self_farming_perception_Z']




In [ ]:
from sklearn.decomposition import PCA
from typing import Literal

def reduce_to_index(
        df: pd.DataFrame,
        cols: list[str],
        index_name: str,
        n_components: int = 1,
        collapse_method: Literal["keep", "weighted-avg"] = "keep"
    ) -> tuple[pd.DataFrame, pd.DataFrame, np.ndarray, np.ndarray]:
    """
    Runs PCA on `cols`, retains the first `n_components`, and either
    keeps them as separate columns or collapses them into one index.

    Parameters
    ----------
    df : original DataFrame (unchanged)
    cols : list of column names to run PCA on
    index_name : base name for new column(s)
    n_components : how many PCs to keep (1 or 2 for your case)
    collapse_method :
        "keep"         -> add PC1, PC2, ... separately
        "weighted-avg" -> single index = Σ w_i * PC_i, w_i = var-ratio_i

    Returns
    -------
    df_out : DataFrame with original cols dropped & new columns added
    loadings_df : each variable’s loading on the retained PCs
    eigenvals : eigenvalues of retained PCs
    var_ratios : proportion of variance explained by each retained PC
    """
    if n_components < 1:
        raise ValueError("n_components must be >= 1")

    # 1. Fit PCA
    pca = PCA(n_components=n_components).fit(df[cols])
    scores      = pca.transform(df[cols])            # shape (n_samples, n_components)
    eigenvals   = pca.explained_variance_            # length = n_components
    var_ratios  = pca.explained_variance_ratio_
    comps       = pca.components_                    # shape (n_components, n_features)

    # 2. Compute loadings for retained PCs
    #    loading_ij = component_ij * sqrt(eigenval_i)
    loadings = comps * np.sqrt(eigenvals[:, None])   # broadcasting
    loadings_df = pd.DataFrame(
        loadings.round(4).T,
        index=cols,
        columns=[f"{index_name}_PC{i+1}" for i in range(n_components)]
    )

    # 3. Build output DataFrame
    df_out = df.drop(columns=cols).copy()
    weights = var_ratios / var_ratios.sum()          # normalise if <1
    composite = (scores * weights).sum(axis=1)
    df_out[index_name] = composite

    return df_out, loadings_df, var_ratios


In [ ]:
corr = nepal_df[target_cols].corr(method="pearson")   # default is Pearson
corr.style.background_gradient(vmin=-1, vmax=1, cmap="coolwarm")  # red-blue scale

In [ ]:
nepal_df, loadings, var = reduce_to_index(nepal_df, target_cols, index_name='Q0__AGR_PROD__continuous', n_components=2)
print(loadings)
print(var)

In [ ]:
corr = senegal_df[target_cols].corr(method="pearson")   # default is Pearson
corr.style.background_gradient(vmin=-1, vmax=1, cmap="coolwarm")  # red-blue scale

In [ ]:
senegal_df, loadings, var = reduce_to_index(senegal_df, target_cols, index_name='Q0__AGR_PROD__continuous', n_components=2)
print(loadings)
print(var)

# Create Resilience Index

Get access to basic services features for each country

In [ ]:
def add_abs_features(
    df: pd.DataFrame,
    lat_col: str = "lat",
    lon_col: str = "lon",
    water_thresh: float = 100,   # meters
    toilet_thresh: float = 50,    # meters
    elec_thresh: float = 100      # meters
) -> pd.DataFrame:
    """
    Enrich any DataFrame with (lat,lon) cols by adding:
      • abs_water_access  (0/1)
      • abs_toilet        (0/1)
      • abs_electricity   (0/1)
    plus these distance columns (in meters):
      dist_water_source, dist_toilet, dist_power_line,
      dist_primary_school, dist_health_facility,
      dist_livestock_market, dist_agri_market,
      dist_public_transport
    """
    # 1. Rename & build GeoDataFrame in EPSG:4326 → project to EPSG:3857 for meters
    df2 = df.rename(columns={lat_col: "lat", lon_col: "lon"}).copy()
    gdf = gpd.GeoDataFrame(
        df2,
        geometry=gpd.points_from_xy(df2.lon, df2.lat),
        crs="EPSG:4326"
    ).to_crs(epsg=3857)

    def add_osm_distance(gdf, tags: dict, col: str):
        # compute lat/lon bbox from projected gdf
        north, south, east, west = gdf.to_crs(epsg=4326).total_bounds[[3,1,2,0]]
        pad = 0.02  # degrees
        north += pad; south -= pad; east += pad; west -= pad
        bbox = (west, south, east, north)  # (left, bottom, right, top)

        # fetch POIs within bbox; signature is features_from_bbox(bbox, tags) :contentReference[oaicite:1]{index=1}
        pois = ox.features_from_bbox(bbox, tags)

        if pois.empty:
            gdf[col] = np.nan
        else:
            # merge all geometries to speed up distance queries
            geom_union = pois.to_crs(epsg=3857).geometry.unary_union
            gdf[col] = gdf.geometry.apply(lambda p: p.distance(geom_union))
        return gdf

    # 2. Compute each service distance
    gdf = add_osm_distance(gdf, {"amenity":"drinking_water"},     "dist_water_source")
    gdf = add_osm_distance(gdf, {"amenity":"toilets"},            "dist_toilet")
    gdf = add_osm_distance(gdf, {"power":"line"},                 "dist_power_line")
    gdf = add_osm_distance(gdf, {"amenity":"school"},             "dist_primary_school")
    gdf = add_osm_distance(gdf, {"amenity":["hospital","clinic"]},"dist_health_facility")
    gdf = add_osm_distance(gdf, {"amenity":"marketplace"},        "dist_livestock_market")
    gdf = add_osm_distance(gdf, {"shop":"greengrocer"},           "dist_agri_market")
    # public transport = nearest of bus_station or bus_stop
    gdf = add_osm_distance(gdf, {"amenity":"bus_station"},        "dist_bus_station")
    gdf = add_osm_distance(gdf, {"highway":"bus_stop"},           "dist_bus_stop")
    gdf["dist_public_transport"] = gdf[["dist_bus_station","dist_bus_stop"]].min(axis=1)

    # 4. Return original + ABS columns
    cols = [
      "dist_water_source","dist_toilet","dist_power_line",
      "dist_primary_school","dist_health_facility",
      "dist_livestock_market","dist_agri_market","dist_public_transport"
    ]
    df2 = df.rename(columns={"lat": lat_col, "lon": lon_col}).copy()
    return pd.concat([df2.reset_index(drop=True), gdf[cols].reset_index(drop=True)], axis=1)

In [ ]:
nepal_df_abs = add_abs_features(
    nepal_df.copy(),
    lat_col="Q1__Latitude__continuous",
    lon_col="Q1__Longitude__continuous"
)

senegal_df_abs = add_abs_features(
    senegal_df.copy(),
    lat_col="Q1__Latitude__continuous",
    lon_col="Q1__Longitude__continuous"
)

Using access to basic services features and other features create a composite sustainable livelihood score for farmers

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


def _ordinal_to_numeric(frame: pd.DataFrame) -> pd.DataFrame:
    """
    Replace every *ordered* categorical column by its `.cat.codes`
    (-1 becomes NaN so it is picked up by the later fillna).
    """
    for col in frame.select_dtypes(include="category").columns:
        if not frame[col].cat.ordered:
            raise TypeError(
                f"Column {col} is categorical but not ordered; "
                "decide whether to one-hot encode, drop, or reorder it."
            )
        # codes: first category→0, second→1, …; missing→-1
        frame[col] = (
            frame[col]
            .cat.codes                     # integers, -1 for NaN
            .replace(-1, pd.NA)            # keep missing values explicit
            .astype("Float64")             # nullable float
        )
    return frame

def compute_sustainable_livelihood_score(
    df: pd.DataFrame,
    country: str
) -> pd.DataFrame:
    """
    Compute a 0–1 'sustainable_livelihood_score' as a formative composite
    across five livelihood domains, using PCA(1+2) per domain where needed.
    """
    df = df.copy()
    country = country.lower()
    
    # ── 1. Domain → feature lists ──────────────────────────────────────────
    if country == "nepal":
        domain_features = {
            "natural": [
                "Q50__How_much_land_that_is_yours_do_you_cultivate_bigha__continuous",
                "Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha_log__continuous",
                "Q52__On_how_much_land_do_you_grow_vegetables_bigha_log__continuous",
                "Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms_log__continuous",
                "Q0__TOTAL_AREA_sqrt__continuous"
            ],
            "physical": [
                "Q55__For_Vegetables_do_you_use_improved_and_or_variety_seeds_yes__binary__1",
                "Q57__If_you_self_prepare_seedlings_how_Nursery__binary__1",
                "Q60__do_you_use_pesticides_or_herbicides_on_this_plot_Yes__binary__1",
                "Q78__How_many_years_have_you_been_using_drip_irrigation_if_no_zero__binary__1",
                "dist_water_source","dist_toilet","dist_power_line",
                "dist_primary_school","dist_health_facility",
                "dist_livestock_market","dist_agri_market","dist_public_transport"
            ],
            "financial": [
                "Q108__What_is_your_households_yearly_income_from_agriculture_NPR_log__continuous",
                "Q109__What_is_your_households_yearly_income_overall_including_agriculture_NPR_log__continuous",
                "Q110__Do_you_currently_have_a_loan_if_no_zero_if_yes_amount_of_money_loaned_NPR__ordinal",
                "Q70__in_the_past_12_months_did_you_receive_any_info_from_anyone_on_agriculture__binary__1"
            ],
            "human": [
                "Q101__how_many_people_live_in_this_household_log__continuous",
                "Q5__AgeYears__continuous",
                "Q106__Education_level_of_this_person_that_is_interviewed_years_of_formal_education__continuous",
                "Q107__Education_level_of_your_wife_husband_How_many_years_of_formal_education_log__continuous"
            ],
            "social": [
                "Q0__hope_total__continuous",
                "Q0__self_control_score__continuous"
            ]
        }

    elif country == "senegal":
        domain_features = {
            "natural": [
                "Q60__Land_owned_cultivated_ha__continuous",
                "Q61__Land_rented_cultivated_ha_log__continuous",
                "Q62__Land_grow_vegetables_ha__continuous",
                "Q71__VEG_harvest_per_year_kg_log__continuous"
            ],
            "physical": [
                "Q69__Use_pesticide_or_herbicide__binary__1",
                "Q84__Years_using_drip_irrigation_log__continuous",
                "Q76__Received_agri_info_last_12m__binary__1",
                "dist_water_source","dist_toilet","dist_power_line",
                "dist_primary_school","dist_health_facility",
                "dist_livestock_market","dist_agri_market","dist_public_transport"
            ],
            "financial": [
                "Q90__Yearly_income_agriculture_XOF_log__continuous",
                "Q91__Yearly_income_overall_XOF_log__continuous",
                "Q92__Current_loan_amount_XOF__ordinal"
            ],
            "human": [
                "Q87__Household_size_sqrt__continuous",
                "Q5__Age__continuous",
                "Q0__Education_years_log__continuous"
            ],
            "social": [
                "Q0__Hope_total__continuous",
                "Q93__Trust_most_people__ordinal"
            ]
        }
    else:
        raise ValueError("country must be 'nepal' or 'senegal'")

    # ── 2. Summarise each domain with PCA(1) or (1+2) ──────────────────────
    domain_evr    = {}
    domain_scores = pd.DataFrame(index=df.index)

    for domain, feats in domain_features.items():
        # a) subset, convert ordinals to numeric, fill, standardise
        X  = _ordinal_to_numeric(df[feats].copy())
        X  = X.fillna(X.mean())                 # now safe: all columns numeric
        Xs = StandardScaler().fit_transform(X)

        # b) fit two PCs and keep 1 or 1+2 as before
        pca                 = PCA(n_components=2, random_state=0).fit(Xs)
        evr1, evr2          = pca.explained_variance_ratio_
        comp1, comp2        = pca.transform(Xs).T
        score               = (evr1 * comp1 + evr2 * comp2) / (evr1 + evr2) \
                              if evr1 < 0.50 else comp1
        domain_evr[domain]  = evr1 + evr2 if evr1 < 0.50 else evr1
        domain_scores[f"{domain}_score"] = score

    # ── 3. Composite: variance‐weighted sum of domain scores ───────────────
    total_evr = sum(domain_evr.values())
    weights   = {d: evr / total_evr for d, evr in domain_evr.items()}

    df["Q0__sustainable_livelihood_score__continuous"] = \
        sum(domain_scores[f"{d}_score"] * w for d, w in weights.items())

    return df["Q0__sustainable_livelihood_score__continuous"], weights


# ── Usage ───────────────────────────────────────────────────────────────
nepal_df["Q0__sustainable_livelihood_score__continuous"], nepal_weights   = compute_sustainable_livelihood_score(nepal_df_abs,   country="nepal")
senegal_df["Q0__sustainable_livelihood_score__continuous"], senegal_weights = compute_sustainable_livelihood_score(senegal_df_abs, country="senegal")

print(nepal_df[["Q0__sustainable_livelihood_score__continuous"]].head())
print(f"{nepal_weights}")
print(senegal_df[["Q0__sustainable_livelihood_score__continuous"]].head())
print(f"{senegal_weights}")


In [ ]:
nepal_df = drop_non_relevant_columns(nepal_df)
senegal_df = drop_non_relevant_columns(senegal_df)

# Category Counts

Get all categorical and binary columns

In [ ]:
cat_cols_nepal = []
cat_cols_senegal = []
for col in nepal_df.columns:
    meta_type = parse_feature_metadata(col)['type']
    if meta_type not in ['continuous', 'discrete', 'time']:
        cat_cols_nepal.append(col)

for col in senegal_df.columns:
    meta_type = parse_feature_metadata(col)['type']
    if meta_type not in ['continuous', 'discrete', 'time']:
        cat_cols_senegal.append(col)

Check that none have an overly large amount of categories

In [ ]:
unique_counts = nepal_df[cat_cols_nepal].nunique()
# 3. see which columns are OK (≤20) or not (>20)
ok      = unique_counts[ unique_counts <= 20 ]
too_many = unique_counts[ unique_counts  > 20 ]

print("Columns with ≤20 categories:\n", ok.sort_values(ascending=False))
print("\nColumns with >20 categories:\n", too_many.sort_values(ascending=False))


In [ ]:
unique_counts = senegal_df[cat_cols_senegal].nunique()
# 3. see which columns are OK (≤20) or not (>20)
ok      = unique_counts[ unique_counts <= 20 ]
too_many = unique_counts[ unique_counts  > 20 ]

print("Columns with ≤20 categories:\n", ok.sort_values(ascending=False))
print("\nColumns with >20 categories:\n", too_many.sort_values(ascending=False))


Only telephone, village and machinery columns have large amounts of categories which makes sense

# Constant Columns

We remove all remaining columns that have only a single value as this adds no new information

In [ ]:
nepal_df = nepal_df.loc[:, nepal_df.nunique(dropna=False) > 1]
senegal_df = senegal_df.loc[:, senegal_df.nunique(dropna=False) > 1]

# Second Clean

Change phone number column to just binary yes no if he has a phone number

In [ ]:

nepal_df['Q4__PhoneNumberAndIsZeroIfNone__nominal'] = (nepal_df['Q4__PhoneNumberAndIsZeroIfNone__nominal'] == 0).astype(int)
senegal_df['Q4__Phone_Number__nominal'] = (senegal_df['Q4__Phone_Number__nominal'] == 0).astype(int)

In [ ]:
nepal_df.rename(columns={'Q4__PhoneNumberAndIsZeroIfNone__nominal' : 'Q4__HasPhoneNumber__binary__1'}, inplace=True)
senegal_df.rename(columns={'Q4__Phone_Number__nominal' : 'Q4__HasPhoneNumber__binary__1'}, inplace=True)


Scale all numeric columns and one hot encode all categorical columns, and deal with missing values in categorical columns by adding a 'missing' category

In [ ]:
# from sklearn.preprocessing import StandardScaler
# import re

# def scale_numeric_columns(df, scale_ordinal=True):
#     df = df.copy()
#     # Get non-categorical numeric columns
#     cols_to_scale = [col for col in df.columns if parse_feature_metadata(col)["type"] in ['continuous', 'discrete', 'ordinal' if scale_ordinal else None]]

#     for col in cols_to_scale:
#         median = df[col].median()
#         # this returns a new Series (up-cast to float64) and assigns it back
#         df[col] = df[col].fillna(median)
    
#     scaler = StandardScaler()
#     df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

#     return df


# def one_hot_encode_columns(df):
#     df = df.copy()
#     cols_to_dummies = []
#     cols_to_dummies_prefixes = []
#     for col in df.columns:
#         meta = parse_feature_metadata(col)
#         if meta["type"] in ['ordinal', 'nominal']:
#             # add it to columns to one hot encode along with its question number
#             cols_to_dummies_prefixes.append(f"{meta['qid']}__{meta['name']}")
#             cols_to_dummies.append((col, meta["type"]))
    
#     col_names_to_dummies, _ = zip(*cols_to_dummies) 
#     col_names_to_dummies = list(col_names_to_dummies)
#     # applying prefix to dummies according to our feature name format
#     dummies = pd.get_dummies(df[col_names_to_dummies], prefix_sep='_', prefix=cols_to_dummies_prefixes, columns=col_names_to_dummies, dtype=int)
    
#     def reformat(col):
#         qid, name = col.split("__")
#         return f"{qid}__{re.sub(r'[^A-Za-z0-9_]', '', name)}"

#     dummies = dummies.rename(columns=reformat)

#     # applying postfix to dummies according to our feature name format
#     rename_map = {}
#     for orig, type in cols_to_dummies:
#         # find all dummy cols that start with "<orig's qid>-<orig's name>"
#         group = [c for c in dummies.columns if c.startswith(f"{parse_feature_metadata(orig)['qid']}__{parse_feature_metadata(orig)['name']}")]
#         # enumerate them 1,2,3…
#         for i, col_name in enumerate(group, start=1):
#             rename_map[col_name] = f"{col_name}__binary_{type}__{i}"
            
#     dummies = dummies.rename(columns=rename_map)
#     df_other = df.drop(columns=col_names_to_dummies)
#     final_df = pd.concat([df_other, dummies], axis=1)
    
#     return final_df

def get_nominals(df):
    cols = []
    for col in df.columns:
        if parse_feature_metadata(col)['type'] == 'nominal':
            cols.append(col)
    return cols

def convert_ordinal_to_numeric(df):
    for col in df.columns:
        if parse_feature_metadata(col)['type'] == 'ordinal' and df[col].map(type).unique()[0] == str:
            if df[col].str.match(r'.*\d$').all():
                df[col] = df[col].apply(lambda x: int(x[-1]))
            elif df[col].cat.ordered:
                df[col] = df[col].cat.codes



Supervisor asked us to not use specific psychological indicators in our final method

In [ ]:


NEPAL_HOPE_QUESTIONS_RANGE = list(range(17,29))
NEPAL_FEELINGS_QUESTIONS_RANGE = list(range(30, 50))
NEPAL_SELF_CONTROL_QUESTIONS_RANGE = list(range(80, 100))
NEPAL_PSYCH_QUESTION_RANGE = NEPAL_HOPE_QUESTIONS_RANGE \
                            + NEPAL_FEELINGS_QUESTIONS_RANGE \
                            + NEPAL_SELF_CONTROL_QUESTIONS_RANGE

SENEGAL_HOPE_QUESTIONS_RANGE = list(range(11,23))
SENEGAL_CHARACTER_STRENGTH_QUESTIONS_RANGE = list(range(24, 60))
SENEGAL_PSYCH_QUESTION_RANGE = SENEGAL_HOPE_QUESTIONS_RANGE \
                                + SENEGAL_CHARACTER_STRENGTH_QUESTIONS_RANGE
# Q51 only used for prod/sust indices, converting to ordinal and dropping log transformed one since produces MAD zero when calculating bicor

NEPAL_ADDITIONAL = ["Q0__positive_total__continuous", "Q0__negative_total__continuous", "Q1__Latitude__continuous", "Q1__Longitude__continuous", "Q1__Accuracy__continuous",
                    "Q78__How_many_years_have_you_been_using_drip_irrigation_if_no_zero__binary__1",
                    "Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha_log__continuous"]
SENEGAL_ADDITIONAL = []
# NEPAL_ADDITIONAL = ["Q1__Latitude__continuous", "Q1__Longitude__continuous", "Q1__Accuracy__continuous", "Q50__How_much_land_that_is_yours_do_you_cultivate_bigha__continuous",
#                     "Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha__continuous", "Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms__continuous",
#                     "Q108__What_is_your_households_yearly_income_from_agriculture_NPR__continuous", "Q52__On_how_much_land_do_you_grow_vegetables_bigha__continuous",
#                     "Q0__positive_total__continuous", "Q0__negative_total__continuous"]
# SENEGAL_ADDITIONAL = ["Q1__Latitude__continuous", "Q1__Longitude__continuous", "Q1__Accuracy__continuous", "Q60__Land_owned_cultivated_ha__continuous", "Q61__Land_rented_cultivated_ha__continuous",
#                       "Q71__VEG_harvest_per_year_kg__continuous", "Q90__Yearly_income_agriculture_XOF__continuous", "Q69__Use_pesticide_or_herbicide__binary__1", "Q0__Distance_Thies_KM__continuous",
#                       "Q0__Distance_Dakar_KM__continuous", "Q62__Land_grow_vegetables_ha__continuous"]

def drop_ordinal_psych_columns(df, country):
    for col in df.columns:
        qid = int(parse_feature_metadata(col)["qid"].replace('Q', ''))
        if country == "NEPAL" and (qid in NEPAL_PSYCH_QUESTION_RANGE or col in NEPAL_ADDITIONAL):
            df.drop(col, axis=1, inplace=True)
        if country == "SENEGAL" and (qid in SENEGAL_PSYCH_QUESTION_RANGE or col in SENEGAL_ADDITIONAL):
            df.drop(col, axis=1, inplace=True)


In [ ]:
# Winsorize continuous features to mitigate extreme outliers
for df in (nepal_df, senegal_df):
    cont_cols = [c for c in df.columns if c.endswith("__continuous")]
    q1 = df[cont_cols].quantile(0.01)
    q99 = df[cont_cols].quantile(0.99)
    df[cont_cols] = df[cont_cols].clip(lower=q1, upper=q99, axis=1)

In [ ]:
convert_ordinal_to_numeric(nepal_df)

drop_ordinal_psych_columns(nepal_df, "NEPAL")
nepal_df.to_excel("nepal_dataframe_FA.xlsx")

In [ ]:
convert_ordinal_to_numeric(senegal_df)

drop_ordinal_psych_columns(senegal_df, "SENEGAL")
senegal_df.to_excel("senegal_dataframe_FA.xlsx")
